# Purpose
This notebook demonstrates the whisky predictions by comparing an input whisky with the recommended whiskies it outputs.

## Load Libraries and Data

In [333]:
import pandas as pd

In [334]:
whiskyt = pd.read_parquet('data/whisky_tfidf.parquet')
similarities = pd.read_parquet('data/similarities2.parquet')
lcbo = pd.read_parquet('../sprint_1/data/lcbo_whisky.parquet')

## Create Dataframe of all Whisky Info
To do this we will add some metrics such as price per 750ml and price per rating

In [335]:
whiskyinfo = (whiskyt.reset_index()
                     .drop({'Name'},axis=1)
                     .set_index('itemnumber')
                     .join(lcbo.set_index('itemnumber'))
                     .assign(price_per_750 = lambda row: row['quantity']*row['price']*750/row['productsize'])
                     .assign(rating_per_dollar_per_750 = lambda row: row['rating_mean']/row['price_per_750'])
            )

whiskyinfo.to_parquet('data/whiskyinfo.parquet')

In [336]:
columns = ['itemname','rating_mean','price','quantity','productsize','price_per_750','rating_per_dollar_per_750']
whiskyinfo[columns].head()

,itemname,rating_mean,price,quantity,productsize,price_per_750,rating_per_dollar_per_750
itemnumber,,,,,,,
619320,12 YO KNAPPOGUE CASTLE IRISH SINGLE MALT WHISKEY,75.100000,73.85,1,750,73.85,1.016926
496729,1792 SINGLE BARREL KENTUCKY STRAIGHT BOURBON W...,72.166667,62.10,1,750,62.10,1.162104
208918,1792 SMALL BATCH KENTUCKY STRAIGHT BOURBON,76.200000,50.20,1,750,50.20,1.517928
634519,601 BOURBON,58.000000,57.95,1,750,57.95,1.000863
255281,ABERFELDY 12 YEAR OLD,76.931034,60.95,1,750,60.95,1.262199


## Functions to Show Matches

In [337]:
# Function to find top matches and get info
def show_top_similarities(itemnumber, top_n=5):
    return (similarities[(similarities['itemnumber'] == itemnumber) & 
             (similarities['itemnumber'] != similarities['itemnumber2'])]
                 .sort_values('sim', ascending=False)
                 .head(top_n)
                 .drop({'nose_sim','taste_sim','finish_sim','itemnumber'},axis=1)
                 .rename({'itemnumber2':'itemnumber','sim':'similarity'},axis=1)
                 .set_index('itemnumber')
                 .join(whiskyinfo)
                 #.drop({'nose','taste','finish'},axis=1)
                 #.rename({'nose_tfidf':'nose','taste_tfidf':'taste','finish_tfidf':'finish'},axis=1)
    )

# Function to find a whiskyID to use in the lookup
def findwhisky(name):
    return whiskyp.reset_index()[whiskyp.reset_index()['Name'].str.contains(name.upper())][['itemnumber','Name']]

# Function to print whisky descriptions:
def printwhiskydesc(itemnumber):
    whisky = whiskyinfo.reset_index()[whiskyt.reset_index()['itemnumber']==itemnumber]
    name = whisky.itemname.values
    price = str(whisky.price.values)
    size = str(whisky.productsize.values)
    rating = str(whisky.rating_mean.values)
    rating_per_dollar_per_750 = str(whisky.rating_per_dollar_per_750.values)
    nose = whisky['nose'].apply(lambda x : ' '.join(x)).values
    taste = whisky['taste'].apply(lambda x : ' '.join(x)).values
    finish = whisky['finish'].apply(lambda x : ' '.join(x)).values
    
    print('Name: ' + name)
    print(' ')
    print('Price: ' + price + ' per ' + size + 'ml')
    print('Rating: ' + rating)
    print('Rating/$/750ml: ' + rating_per_dollar_per_750)
    print(' ')
    print('Nose: ' + nose)
    print(' ')
    print('Taste: ' + taste)
    print(' ')
    print('Finish: ' + finish)
    print(' ')
    
# Function to show all suggestions and details of the top suggestion
def displaysuggestions(itemnumber, sort='rating_per_dollar_per_750', top_n = 5):
    columns = ['itemname',
              'similarity', 'rating_per_dollar_per_750','rating_mean','price','productsize',
              'style',
               'nose','taste','finish'
              ]
    results = show_top_similarities(itemnumber, top_n)[columns].sort_values(sort,ascending=False)
    # Print main results
    display(results)
    # Print info about search whisky
    printwhiskydesc(itemnumber)
    # Print info about top match whisky
    printwhiskydesc(results.reset_index().itemnumber.iloc[0])

## Test Suggestions

### Islay Scotch: Laphroaig 10

In [338]:
# Find a whisky ID
findwhisky('Laphroaig 10')

,itemnumber,Name
265,248997,LAPHROAIG 10 YEAR OLD ISLAY SINGLE MALT SCOTCH...


In [352]:
displaysuggestions(248997, sort='similarity')

,itemname,similarity,rating_per_dollar_per_750,rating_mean,price,productsize,style,nose,taste,finish
itemnumber,,,,,,,,,,
560474,ARDBEG 10 YEAR OLD ISLAY SCOTCH WHISKY,0.736804,0.859718,86.143750,100.20,750,Islay,"[peat, smoke, sea, brine, iodine, campfire, me...","[peat, smoke, medicinal, like, sweetness, iodi...","[smoke, peat, linger, taste, peaty, like, mout..."
19158,LAPHROAIG QUARTER CASK ISLAY SINGLE MALT SCOTC...,0.719129,0.976320,87.478261,89.60,750,Islay,"[peat, smoke, medicinal, iodine, laphroaig, se...","[peat, smoke, medicinal, iodine, brine, salt, ...","[smoke, peat, medicinal, iodine, linger, dry, ..."
207126,LAGAVULIN 16 YEAR OLD ISLAY SINGLE MALT SCOTCH...,0.712917,0.644943,87.196319,135.20,750,Islay,"[peat, smoke, iodine, campfire, medicinal, she...","[peat, smoke, like, iodine, sherry, brine, sal...","[smoke, peat, smoky, linger, lingers, taste, s..."
249680,TALISKER 10 YEAR OLD SINGLE MALT SCOTCH WHISKY,0.708168,0.816912,85.939189,105.20,750,Island,"[smoke, peat, sea, brine, salt, pepper, salty,...","[smoke, peat, salt, brine, oily, salty, sea, s...","[smoke, peat, pepper, salt, linger, dry, warm,..."
455436,LAPHROAIG LORE ISLAY SINGLE MALT SCOTCH WHISKY,0.675212,0.432398,84.641975,195.75,750,Islay,"[peat, medicinal, smoke, laphroaig, seaweed, i...","[smoke, peat, medicinal, ash, iodine, brine, l...","[smoke, peat, ash, proof, peaty, ashy, sweetne..."


['Name: LAPHROAIG 10 YEAR OLD ISLAY SINGLE MALT SCOTCH WHISKY']
 
Price: [74.95] per [750]ml
Rating: [84.79452055]
Rating/$/750ml: [1.13134784]
 
['Nose: peat smoke iodine medicinal sea salt seaweed campfire laphroaig brine smell burning like band meat rubber bacon wet coal wood toffee strong salty ash medicine charcoal sweetness air slight citrus leather earthy lemon beach immediately earth hint ocean aid water burnt flavor big tar moss face sweeter old present come']
 
['Taste: smoke peat iodine sweetness meat salt ash medicinal sea brine seaweed like campfire nose oily rubber coal little bacon smokey mouthfeel peaty water salty hint slight feel wood earthy rich tongue flavor dry smooth laphroaig grass citrus ashy damp char smoky strong mouth smokiness wet leather tobacco peppery malt smell']
 
['Finish: smoke peat medicinal iodine smoky smokey linger sweetness taste like salt stay mouth earthy peaty note flavor ash burn tar feel little salty smooth fade good laphroaig warm bit hint 

### Bourbon:  Wild Turkey Rare Breed

In [356]:
findwhisky('Bocan')

,itemnumber,Name
117,578351,CU BOCAN
387,421719,TOMATIN CU BOCAN HIGHLAND SINGLE MALT SCOTCH W...


In [358]:
displaysuggestions(421719)

,itemname,similarity,rating_per_dollar_per_750,rating_mean,price,productsize,style,nose,taste,finish
itemnumber,,,,,,,,,,
477083,BENROMACH ORGANIC SPEYSIDE,0.527288,0.758491,75.375000,92.75,700,Speyside,"[aromatic, lemon, liqueur, wax, sour, sherry, ...","[intense, spicy, ash, creamy, wax, lime, nose,...","[ash, creamy, type, peat, sour, basically, mar..."
578369,GLENDRONACH PEATED PORT WOOD FINISH SINGLE MALT,0.542462,0.735165,82.666667,104.95,700,Highland,"[juicy, emerge, cumin, paprika, earthy, peat, ...","[ish, smoke, layer, chocolate, meat, chili, ti...","[red, chili, powder, peach, tannic, wine, toba..."
435974,THE DEVERON 18 YEAR OLD HIGHLAND SINGLE MALT S...,0.534678,0.464947,64.000000,137.65,750,Highland,"[white, yellow, golden, delicate, coconut, pin...","[vaguely, best, arrives, despite, beer, know, ...","[coastal, strawberry, char, tobacco, fruity, c..."
546101,COLLECTIVUM XXVIII,0.528005,0.316347,86.600000,273.75,750,Blend,"[look, come, chocolate, smoke, creamy, tobacco...","[savory, dark, black, allspice, apples, ham, p...","[neat, peat, lemon, smoke, tea, pepper, chocol..."
578310,BLACK ART 6.1,0.527396,0.237255,91.500000,359.95,700,Islay,"[coconut, hazelnut, start, mango, smell, raspb...","[richness, cooky, spread, unique, mocha, yeast...","[cedar, maybe, savoury, seed, briny, cumin, li..."


['Name: TOMATIN CU BOCAN HIGHLAND SINGLE MALT SCOTCH WHISKY']
 
Price: [86.] per [750]ml
Rating: [84.5]
Rating/$/750ml: [0.98255814]
 
['Nose: acrid floor layer forest blueberry underneath fudge tart earthy almond peat maple rich syrup malt pear cherry']
 
['Taste: tartness polish arrives black layer wood build development pleasant warm old tea peat fruity oily lot cinnamon']
 
['Finish: berry char earthy toffee orange chocolate honey linger']
 
['Name: BENROMACH ORGANIC SPEYSIDE']
 
Price: [92.75] per [700]ml
Rating: [75.375]
Rating/$/750ml: [0.75849057]
 
['Nose: aromatic lemon liqueur wax sour sherry pepper red malt fig fresh banana cherry orange gooseberry noticeably pledge zesty pod wine white nutty amaze peat nutmeg room lemongrass combo acidic aniseed smith style granny maraschino dark syrup rotten chocolate citrusy second blood burning highland golden meaty ash yeasty nose hint solvent']
 
['Taste: intense spicy ash creamy wax lime nose orange crisp peat sour mango red great co

### Unconventional Islay Scotch: Bruichladdich Classic Laddie

In [342]:
findwhisky('Bruichladdich')

,itemnumber,Name
61,459024,BRUICHLADDICH ISLAY BARLEY 2009
62,387423,BRUICHLADDICH THE CLASSIC LADDIE SCOTTISH BARLEY
63,551820,BRUICHLADDICH THE ORGANIC 2009


In [343]:
displaysuggestions(387423)

,itemname,similarity,rating_per_dollar_per_750,rating_mean,price,productsize,style,nose,taste,finish
itemnumber,,,,,,,,,,
192732,CRAGGANMORE 12 YEAR OLD SINGLE MALT SCOTCH WHISKY,0.633632,1.186513,79.021739,66.60,750,Speyside,"[pear, floral, malt, almond, dry, subtle, smok...","[smoke, almond, creamy, malty, salty, peat, nu...","[smoke, bit, honey, light, linger, hint, dry, ..."
515346,OLD PULTENEY 12 YEAR OLD HIGHLAND SINGLE MALT ...,0.630943,1.080579,80.985507,69.95,700,Highland,"[sea, brine, salt, floral, pear, air, lemon, a...","[salt, salty, citrus, sea, lemon, brine, toffe...","[salty, salt, fruit, briny, vanilla, citrus, f..."
459024,BRUICHLADDICH ISLAY BARLEY 2009,0.650515,1.060742,84.806306,79.95,750,Islay,"[peat, smoke, lemon, salt, sea, salty, fresh, ...","[peat, smoke, salt, salty, lemon, barley, malt...","[peat, smoke, dry, salty, honey, bit, vanilla,..."
250357,CLYNELISH 14 YEAR OLD SINGLE MALT SCOTCH WHISKY,0.634424,0.921000,81.876923,88.90,750,Highland,"[candle, wax, orange, floral, malt, citrus, fr...","[malt, smoke, mouthfeel, orange, feel, little,...","[dry, waxy, smoke, pepper, malt, honey, pineap..."
241943,GLENMORANGIE LASANTA EXTRA MATURED HIGHLAND SI...,0.637842,0.838834,77.885714,92.85,750,Highland,"[sherry, orange, raisin, dry, wine, floral, ci...","[sherry, orange, raisin, wine, like, nose, sug...","[sherry, fruit, spice, pepper, honey, bitter, ..."


['Name: BRUICHLADDICH THE CLASSIC LADDIE SCOTTISH BARLEY']
 
Price: [70.25] per [750]ml
Rating: [81.97368421]
Rating/$/750ml: [1.16688518]
 
['Nose: smoke peat fresh floral wine bubble barley melon gum salty malt dry faint salt plum peach sea pineapple pear raspberry lemon cut honeydew citrus strawberry like funk chocolate little cream subtle grass slight maybe zest raisin red crisp coastal malty white spice fig blueberry ripe cherry highland small scent wet']
 
['Taste: salt salty lemon mint sugar smoke malt brine barley floral sea bubble pear melon grape gum chocolate toast heat nice peat nectarine delicious green lot raisin banana orange cocoa cherry pith spicy malty peach citrus coconut pineapple sweetness butter nose sugary clean raw like red powder brown tounge ton prune']
 
['Finish: smoke linger salty warm honey vanilla note apple peat heat faint acidic ice burn barbecue dry accompany white port bit sugar pepper lime sauce cherry orange little sweetness hay light malt herbal st

### Irish: Redbreast 15

In [344]:
findwhisky('Redbreast')

,itemnumber,Name
313,446781,REDBREAST 15 YEAR OLD IRISH WHISKEY
314,446799,REDBREAST 21 YEAR OLD IRISH WHISKEY
315,481218,REDBREAST LUSTAU EDITION IRISH WHISKEY


In [345]:
displaysuggestions(446781)

,itemname,similarity,rating_per_dollar_per_750,rating_mean,price,productsize,style,nose,taste,finish
itemnumber,,,,,,,,,,
481218,REDBREAST LUSTAU EDITION IRISH WHISKEY,0.607529,0.901673,79.933333,88.65,750,Ireland,"[sherry, raisin, accord, wife, cinnamon, usual...","[sherry, slightly, creamy, marzipan, damn, pea...","[bit, warmth, spice, sherry, surprised, little..."
446807,YELLOW SPOT IRISH WHISKEY,0.617304,0.871204,85.857143,98.55,750,Ireland,"[grape, dense, nice, fruity, red, subtle, abv,...","[sherry, copper, red, mature, nice, malt, port...","[copper, bit, note, vanilla, pretty, tad, high..."
221200,THE BALVENIE CARIBBEAN CASK 14 YEAR OLD SCOTCH...,0.596704,0.597028,83.703297,140.20,750,Speyside,"[rum, sugar, spice, smell, brown, tropical, ba...","[rum, sugar, cinnamon, sweetness, tropical, br...","[rum, spice, smooth, note, sugar, vanilla, lik..."
446799,REDBREAST 21 YEAR OLD IRISH WHISKEY,0.614801,0.358352,87.545455,244.30,750,Ireland,"[golden, concentrate, old, pineapple, wood, ma...","[raspberry, cream, pear, join, malt, rich, min...","[mango, spice, barley, pineapple, pepper, clov..."
446757,MIDLETON BARRY CROCKETT LEGACY SINGLE POT STILL,0.619780,0.290200,87.800000,302.55,750,Ireland,"[pot, peach, single, grassy, creamy, mention, ...","[complexity, tea, port, cinnamon, sherry, red,...","[tad, harshness, sandalwood, floral, thyme, de..."


['Name: REDBREAST 15 YEAR OLD IRISH WHISKEY']
 
Price: [122.8] per [750]ml
Rating: [85.55555556]
Rating/$/750ml: [0.69670648]
 
['Nose: sherry stay accord wife finish malt floral creamy score fresh toffee apricot cinnamon almond close biscuit hazelnut spice metallic rich yeast come raisin somewhat dry banana licorice influence acetone raspberry spicy hint calm power head batter main milder pineapple flour previous pot play malty bourbon fantastic ginger esque solventy mint']
 
['Taste: malt sherry cream creamy oily water pie raspberry fresh rich sugar slightly bland red near brown come bean port waxy begin mild flavour crisp metallic mint butter raisin alcohol smooth texture syrup round flavor nice hint add warm like cinnamon talk its echo bread generic way pretty apricot grass curry']
 
['Finish: bit spice chip peel fresh aftertaste red apple light water tad pineapple pepper vanilla lemon rye whisky corn little like cinnamon grape wood leather leaf note harshness sherry way dry slight

### Japanese: Toki

In [346]:
findwhisky('Toki')

,itemnumber,Name
383,465484,TOKI


In [347]:
displaysuggestions(465484, sort='similarity')

,itemname,similarity,rating_per_dollar_per_750,rating_mean,price,productsize,style,nose,taste,finish
itemnumber,,,,,,,,,,
481226,HIBIKI HARMONY,0.646540,0.656156,78.705882,119.95,750,Japan,"[distinctive, grain, spice, place, fresh, whit...","[grain, sherry, pear, blend, touch, lightly, s...","[grass, waxy, tree, grain, honey, apple, light..."
238097,DALWHINNIE 15 YEAR OLD SINGLE HIGHLAND MALT SC...,0.589329,0.774003,81.270270,105.00,750,Highland,"[floral, smoke, hint, malt, orange, citrus, he...","[smoke, malt, like, water, citrus, floral, pea...","[smoke, honey, linger, malt, flavor, dry, bit,..."
289496,CARDHU 12 YEAR OLD SINGLE MALT SCOTCH WHISKY,0.588885,0.980196,76.406250,77.95,750,Speyside,"[smoke, malt, pear, bran, peat, barley, wood, ...","[smoke, peat, salt, slight, oily, like, pear, ...","[bit, dry, linger, pepper, peat, smoke, heat, ..."
69435,GLENKINCHIE 12 YEARS OLD LOWLAND SINGLE MALT S...,0.588189,0.869743,76.972222,88.50,750,Lowland,"[floral, lemon, pear, malt, wood, malty, green...","[smoke, citrus, malty, white, lemon, creamy, l...","[burn, dry, taste, white, bit, sour, cereal, c..."
250357,CLYNELISH 14 YEAR OLD SINGLE MALT SCOTCH WHISKY,0.581881,0.921000,81.876923,88.90,750,Highland,"[candle, wax, orange, floral, malt, citrus, fr...","[malt, smoke, mouthfeel, orange, feel, little,...","[dry, waxy, smoke, pepper, malt, honey, pineap..."


['Name: TOKI']
 
Price: [54.95] per [750]ml
Rating: [71.5]
Rating/$/750ml: [1.30118289]
 
['Nose: pear white grape floral green gentle brine citrus flower spice sweat wine malty sock faint honeycomb aspect grass base characteristic harsh fruity barely smoke definite banana bouquet hint delicate wheat simple different mix iodine crisp burn follow somewhat rum influence grassy melon tea coconut reminds oaky hay old apricot ginger']
 
['Taste: dryness grain pear like main oaky white slight floral odd tea grass tongue bitterness whisky fresh tingle nice grassy blend taste reminds ginger fairly mouthfeel juice spread apples trail general malty bar hot glaze chocolatey sherry trace cream honeysuckle element skin creamy continue pith layer weird seed left classic citrus']
 
['Finish: clean like waxy linger dry light white juice grape grain aftertaste good citrusy tone ginger flat plastic cranberry smoke hard sweetness build crisp grapefruit honey hay blend ashy sharp bit syrupy lightly medici

### Canadian: Crown Royal

In [348]:
findwhisky('Crown')

,itemnumber,Name
108,403519,CROWN ROYAL APPLE
109,224352,CROWN ROYAL BLACK
110,276618,CROWN ROYAL BLACK WHISKY
111,246603,CROWN ROYAL LIMITED EDITION WHISKY
112,312801,CROWN ROYAL MAPLE FINISHED
113,434092,CROWN ROYAL NORTHERN HARVEST RYE
114,321208,CROWN ROYAL SPECIAL RESERVE
115,1487,CROWN ROYAL WHISKY
116,438911,CROWN ROYAL XO


In [349]:
displaysuggestions(1487)

,itemname,similarity,rating_per_dollar_per_750,rating_mean,price,productsize,style,nose,taste,finish
itemnumber,,,,,,,,,,
434092,CROWN ROYAL NORTHERN HARVEST RYE,0.609016,2.167031,77.904762,35.95,750,Canada,"[rye, smell, spice, clove, like, hint, generic...","[rye, canadian, whisky, like, butterscotch, hi...","[flavor, warmth, allspice, linger, fading, rye..."
405316,W.L. WELLER SPECIAL RESERVE BOURBON,0.610109,1.945961,78.422222,40.30,750,Bourbon,"[cinnamon, alcohol, sugar, brown, sweetness, l...","[cinnamon, smooth, entry, sugar, sweetness, bu...","[burn, cinnamon, bit, spice, vanilla, aftertas..."
482729,BULLEIT BOURBON,0.607167,1.709238,80.163265,23.45,375,Bourbon,"[rye, spice, cinnamon, dill, bourbon, corn, su...","[rye, cinnamon, mint, corn, spicy, bourbon, ni...","[rye, mint, spice, spicy, dry, mellow, fairly,..."
246603,CROWN ROYAL LIMITED EDITION WHISKY,0.627793,1.647232,68.277778,41.45,750,Canada,"[maple, rye, bourbon, prickle, peach, wife, sy...","[rye, canadian, brings, juniper, maple, whisky...","[note, syrup, vanilla, nice, unfortunately, pe..."
321208,CROWN ROYAL SPECIAL RESERVE,0.636648,1.127606,67.600000,59.95,750,Canada,"[maple, rye, alcohol, sugar, sort, bourbon, gr...","[rye, batch, royal, canadian, maple, course, t...","[note, syrup, maple, flavour, vanilla, nice, a..."


['Name: CROWN ROYAL WHISKY']
 
Price: [30.5] per [750]ml
Rating: [74.72972973]
Rating/$/750ml: [2.45015507]
 
['Nose: maple rye pancake like syrup bourbon cola smell corn sweetness aroma little alcohol acetone clove gum overwhelm appeal green woodiness banana present grain particular butterscotch overall unique butter cinnamon sugar hard detect peach oatmeal wife thought whisky reminds fragrant cherry ethanol floral juicy lighter earthy especially nose mint high solvent']
 
['Taste: rye maple royal canadian whisky butterscotch cinnamon classic mouth juniper definite typical buttercream caraway bold mouthfeel grain like bitterness come cedar subdue feel smooth syrup night end ethanol corn join complexity holy summer cola hint ice nose detect butter wonderful clove sip sticky quickly tip peanut character lingers cherry pith']
 
['Finish: butterscotch note linger bitterness cola proof rye vanilla wine caramel unfortunately high spice syrup little maple light thought palate classic honestl